<a href="https://colab.research.google.com/github/Divya-demo1/Deep-Learning-Projects/blob/master/Quora_project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()
import os

**Loading file from dropbox**


In [0]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-02-06 22:55:00--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-02-06 22:55:00--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce2b6abbbe64ac9f77f4160d64d.dl.dropboxusercontent.com/zip_download_get/AVDRF7Z9zoEOpkUYVCVcLoSpZI4ielprOvDnjWmaQIrteuSdWWQfehy3uLvO7tIo11G_ww1VJpQroUHP_x9eZeCrvTYoQXst_wQEkU_9KeNpIQ [following]
--2020-02-06 22:55:00--  https://uce2b6abbbe64ac9f77f4160d64d.dl.dropboxusercontent.com/zip_download_get/AVDRF7Z9zoEOpkUYVCVcLoSpZI4ielprOvDnjWmaQIrteuSdWWQfehy3uLvO7tIo11G_ww1VJpQroUHP_x9e

In [0]:
!unzip AAAw1_JIzpuVvwteJCma0xMla?dl=0

Archive:  AAAw1_JIzpuVvwteJCma0xMla?dl=0
mapname:  conversion of  failed
 extracting: train.csv               

1 archive had fatal errors.


In [0]:
os.listdir()

['.config', 'AAAw1_JIzpuVvwteJCma0xMla?dl=0', 'train.csv', 'sample_data']

In [0]:

import sklearn.datasets as skds
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [0]:
quora = pd.read_csv('train.csv',encoding = 'ISO-8859-1')

In [0]:
quora.head()

In [0]:
quora.shape

(1306122, 3)

In [0]:
quora.drop(['qid'],inplace=True,axis=1)

In [0]:

quora.head()

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0


In [0]:
quora_train,quora_test=train_test_split(quora,test_size=0.2,random_state=2)

In [0]:
x_train=quora_train['question_text']
y_train=quora_train['target']
x_test=quora_test['question_text']
y_test=quora_test['target']

In [0]:
import nltk

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
sent_lens=[]
for sent in quora_train['question_text']:
    sent_lens.append(len(word_tokenize(sent)))

In [0]:

max(sent_lens)

412

In [0]:
max_quant_len=np.quantile(sent_lens,0.95)

In [0]:
max_quant_len=int(max_quant_len)
max_quant_len

30

In [0]:
max_len =max_quant_len
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(x_train)
sequences_train = tok.texts_to_sequences(x_train)

In [0]:
#tok.index_word

In [0]:
vocab_len=len(tok.index_word.keys())

In [0]:
vocab_len

196220

In [0]:
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_quant_len)

In [0]:
sequences_matrix_train

array([[   0,    0,    0, ..., 5916,   44,  242],
       [   0,    0,    0, ...,    5, 1602, 3144],
       [   0,    0,    0, ...,  136,   18,  296],
       ...,
       [   0,    0,    0, ...,  392,    7,  678],
       [   0,    0,    0, ...,    6,   28,  560],
       [   0,    0,    0, ..., 5472,    6,  142]], dtype=int32)

**Loading data from google drive**

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: stacksample.zip, id: 12vawrfGZCl3CWFDSV7_6OTzwOKckcvSR
title: glove.6B.100d.txt.zip, id: 1UuFu2iM81r-zCTELWKcT4zY2QRTpijCo
title: train.csv, id: 1h_QAPLzFre6QVRZ7Bu7DIel6yaArAKNh
title: Colab Notebooks, id: 1ATgDoTZH_i3iMLX3Cs56sBZ0rAkqiPbg
title: New folder, id: 1Mgd5DdBB1b0kJ02Q7WARnxcD3tpDrR8K
title: Python_basics_class_assign.ipynb, id: 0B139lAmq78gnbGZaSElPaFNRRkFiRFd4UXIzTEVzXy1oRk5z
title: diabetes.csv, id: 0B139lAmq78gnOC1SQ3NxVkZJcWR3aXltU3IzYk04YXUyR0Rv
title: Untitled presentation, id: 18wTMHUG4aHkXQNshqC4QEUFrbh4dfos7dQuopy2ubDo
title: icicijulytodecember, id: 1xuo0sljCGw3LlM2-8GVWK-wee7uGFPrBmgPaUUjl60U
title: DEEPTHI Resume2, id: 1SR4_8wBTdeuWHCiTHoNt668XBYHOuHlk_RtYc2veR_A
title: Untitled document, id: 1jfbe_2mfK7ktxIhckuzCnAJ7z_W0vWfzN7b0xfx7XsU
title: summary, id: 1oXCb8J6hUX_JCLeDw58zuiYctsoNVH1Kn1J3NkdtjRs
title: Digit Per Minute(DPM) (1), id: 1H8oLNZAKMdZpvmKeOQKSgxP6UUG_4j24JryoG7Yvf8o
title: 12day shudule children video, id: 1f8SCzRlgrNnvvL_hbiYG665pZxuUSeD

In [0]:
downloaded = drive.CreateFile({'id': '1UuFu2iM81r-zCTELWKcT4zY2QRTpijCo'})
downloaded.GetContentFile('glove6b100dtxt.zip')

In [0]:
!unzip glove6b100dtxt.zip

Archive:  glove6b100dtxt.zip
  inflating: glove.6B.100d.txt       


In [0]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

 
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')

In [0]:
embeddings_index = {}
f = open('glove.6B.100d.txt',encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
word_index=tok.word_index
len(word_index)

196220

In [0]:
embedding_matrix = np.zeros((len(word_index)+1,100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       [-0.15180001,  0.38409001,  0.89340001, ..., -0.27123001,
         0.22157   ,  0.92111999],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.067863  , -0.26969001, -0.36182001, ...,  0.16723999,
        -0.08863   ,  0.17496   ],
       [ 0.046097  ,  0.45210001,  0.1778    , ...,  0.10402   ,
         0.029458  ,  0.52072001]])

In [0]:

tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(x_test)
sequences_test = tok.texts_to_sequences(x_test)

In [0]:
#sequences_test

In [0]:
sequences_matrix_test = sequence.pad_sequences(sequences_test,maxlen=max_quant_len)

In [0]:
sequences_matrix_test

array([[   0,    0,    0, ...,   61,   94, 4200],
       [   0,    0,    0, ...,  118,   28,  142],
       [   0,    0,    0, ...,    3,   64,  888],
       ...,
       [   0,    0,    0, ...,  215, 5927, 6025],
       [   0,    0,    0, ...,  395,  128, 1193],
       [ 124,  114,    1, ..., 3896, 7377, 1418]], dtype=int32)

In [0]:
x_train=sequence.pad_sequences(sequences_train,maxlen=max_quant_len)

In [0]:
x_test=sequence.pad_sequences(sequences_test,maxlen=max_quant_len)

In [0]:
from keras.layers import Embedding

In [0]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding

In [0]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len], dtype='float32')
    layer = Embedding(len(word_index)+1,100,input_length=max_len,weights=[embedding_matrix],mask_zero=True,trainable=False)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)

    layer = Dense(128,name='FC2')(layer)
    layer = Activation('tanh')(layer)
    layer = Dropout(0.5)(layer)

    layer = Dense(256,name='FC3')(layer)
    layer = Activation('tanh')(layer)
    layer = Dropout(0.5)(layer)
    #layer=BatchNormalization()(layer)
    
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [0]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 30.0)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 30.0, 100)         19622100  
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                42240     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_6 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
FC2 (Dense)                  (None, 128)               3289

In [0]:
model.fit(x_train, y_train,
          batch_size=10000,
          epochs=15,
          validation_data=(x_test, y_test))

Train on 1044897 samples, validate on 261225 samples
Epoch 1/15
1044897/1044897 [==============================] - 372s 356us/step - loss: 0.1184 - acc: 0.9535 - val_loss: 0.2684 - val_acc: 0.9348
Epoch 2/15
1044897/1044897 [==============================] - 368s 352us/step - loss: 0.1165 - acc: 0.9542 - val_loss: 0.2487 - val_acc: 0.9288
Epoch 3/15
1044897/1044897 [==============================] - 364s 348us/step - loss: 0.1150 - acc: 0.9549 - val_loss: 0.2491 - val_acc: 0.9323
Epoch 4/15
1044897/1044897 [==============================] - 364s 348us/step - loss: 0.1134 - acc: 0.9554 - val_loss: 0.2578 - val_acc: 0.9322
Epoch 5/15
1044897/1044897 [==============================] - 362s 347us/step - loss: 0.1118 - acc: 0.9559 - val_loss: 0.2552 - val_acc: 0.9288
Epoch 6/15
1044897/1044897 [==============================] - 364s 348us/step - loss: 0.1110 - acc: 0.9562 - val_loss: 0.2709 - val_acc: 0.9336
Epoch 7/15
1044897/1044897 [==============================] - 365s 349us/step - los

In [0]:
predictions=model.predict(sequences_matrix_test)

In [0]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,predictions)

0.7182648653158381

In [0]:
#Obseravations:
#when epochs=10000 roc_auc_score is 0.7173321158355012 but the model is overfitting (val_acc<acc)
#when only 1 dense layer and a dropout layer is used roc_auc_score 0.7292734137923427 
#when 3 dense layers and a dropout layer is used roc_auc_score 0.7271602252073571
#as the loss is decreasing it implies the model is learning so i increased no of epochs to 15 but val_loss increased 